In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/'My Drive'
!mkdir NMT

/content/drive/My Drive


In [3]:
%cd NMT

/content/drive/My Drive/NMT


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [5]:
!wget http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz
!tar -xvzf training-parallel-nc-v13.tgz

--2020-02-26 13:15:33--  http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113157482 (108M) [application/x-gzip]
Saving to: ‘training-parallel-nc-v13.tgz’

training-parallel-n 100%[===================>] 107.92M  76.4MB/s    in 1.4s    

2020-02-26 13:15:35 (76.4 MB/s) - ‘training-parallel-nc-v13.tgz’ saved [113157482/113157482]

training-parallel-nc-v13/
training-parallel-nc-v13/news-commentary-v13.ru-en.ru
training-parallel-nc-v13/news-commentary-v13.cs-en.en
training-parallel-nc-v13/news-commentary-v13.de-en.de
training-parallel-nc-v13/news-commentary-v13.ru-en.en
training-parallel-nc-v13/news-commentary-v13.zh-en.zh
training-parallel-nc-v13/news-commentary-v13.zh-en.en
training-parallel-nc-v13/news-commentary-v13.cs-en.cs
training-parallel-nc-v13/news-commentary

In [6]:
!head training-parallel-nc-v13/news-commentary-v13.ru-en.ru

Золото по 10 000 долларов?
САН-ФРАНЦІСКО. Разговор о стоимости золота редко получается рациональным, тем более в последнее время, так как цены на золото выросли более чем на 300% за десятилетие.
Еще в декабре прошлого года экономисты-коллеги Мартин Фельдштейн и Нуриэль Рубини опубликовали свои пророческие статьи в колонках альтернативных мнений, храбро ставя в них под вопрос стремление игры на повышение, благоразумно указывая на риски золота.
И что бы вы думали?
С тех пор как вышли их статьи, стоимость золота повысилась еще больше.
Недавно цена на золото даже достигла рекордной отметки в 1 300 долларов за унцию.
В декабре прошлого года многие сторонники сохранения денежных функций золота утверждали, что его цена неизбежно дойдет до 2 000 долларов.
Теперь, воодушевленные постоянным ростом цен, некоторые считают, что цена на золото может вырасти еще сильнее.
Один успешный инвестор в золото недавно объяснил мне, что цена акций падала в течение более десятилетия, прежде чем в начале 1980-х

In [7]:
!head training-parallel-nc-v13/news-commentary-v13.ru-en.en

$10,000 Gold?
SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.
Lately, with gold prices up more than 300% over the last decade, it is harder than ever. Just last December, fellow economists Martin Feldstein and Nouriel Roubini each penned op-eds bravely questioning bullish market sentiment, sensibly pointing out gold’s risks.
Wouldn’t you know it?
Since their articles appeared, the price of gold has moved up still further.
Gold prices even hit a record-high $1,300 recently.
Last December, many gold bugs were arguing that the price was inevitably headed for $2,000.
Now, emboldened by continuing appreciation, some are suggesting that gold could be headed even higher than that.
One successful gold investor recently explained to me that stock prices languished for a more than a decade before the Dow Jones index crossed the 1,000 mark in the early 1980’s.
Since then, the index has climbed above 10,000.


In [0]:
!head -n 99000 training-parallel-nc-v13/news-commentary-v13.ru-en.en > training-parallel-nc-v13/news-commentary-v13.ru-en.train.en
!head -n 99000 training-parallel-nc-v13/news-commentary-v13.ru-en.ru > training-parallel-nc-v13/news-commentary-v13.ru-en.train.ru
!tail -n 1000 training-parallel-nc-v13/news-commentary-v13.ru-en.en > training-parallel-nc-v13/news-commentary-v13.ru-en.valid.en
!tail -n 1000 training-parallel-nc-v13/news-commentary-v13.ru-en.ru > training-parallel-nc-v13/news-commentary-v13.ru-en.valid.ru

In [0]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [0]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [0]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [0]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [0]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [0]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [0]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [0]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [0]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [0]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [0]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [0]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [0]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [0]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [0]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [0]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [25]:
tmp_model = make_model(10, 10, 2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [26]:
tmp_model

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()

In [0]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [0]:
import torch
def run_epoch(data_iter, model, loss_compute, path):
    "Standard Training and Logging Function"
    if path==None:
      path = ('ru_en_nmt_model','ru_en_nmt_opt')
    else:
      model.load_state_dict(torch.load(path[0]))
      loss_compute.opt.optimizer.load_state_dict(torch.load(path[1]))
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src.cuda(), batch.trg.cuda(), 
                            batch.src_mask.cuda(), batch.trg_mask.cuda())
        loss = loss_compute(out.cuda(), batch.trg_y.cuda(), batch.ntokens.cuda())
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
        if i % 500 == 1:
            print('Save model and optimizer. Chpoint step #{}'.format(i))
            torch.save(model.state_dict(), path[0])
            torch.save(loss_compute.opt.optimizer.state_dict(), path[1])

    return total_loss / total_tokens

In [0]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [0]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [0]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data * norm

In [33]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 8.8MB/s 


In [0]:
from torchtext import data, datasets
import sentencepiece as spm

spm.SentencePieceTrainer.Train('--input=training-parallel-nc-v13/news-commentary-v13.ru-en.train.ru --model_prefix=bpe_ru --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')
tok_ru = spm.SentencePieceProcessor()
tok_ru.load('bpe_ru.model')

SRC = data.Field(pad_token="<blank>", fix_length=50,init_token='<s>', eos_token='</s>', 
                 tokenize = lambda x: tok_ru.encode_as_pieces(x), lower=True)

spm.SentencePieceTrainer.Train('--input=training-parallel-nc-v13/news-commentary-v13.ru-en.train.en --model_prefix=bpe_en --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')
tok_en = spm.SentencePieceProcessor()
tok_en.load('bpe_en.model')

TGT = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_en.encode_as_pieces(x),  is_target=True, unk_token=None
)

In [0]:
valid = datasets.TranslationDataset(path = 'training-parallel-nc-v13/news-commentary-v13.ru-en.valid', exts=('.ru', '.en'), fields=(SRC, TGT))
train = datasets.TranslationDataset(path = 'training-parallel-nc-v13/news-commentary-v13.ru-en.train', exts=('.ru', '.en'), fields=(SRC, TGT))

SRC.build_vocab(train.src, min_freq=2)
TGT.build_vocab(train.trg, min_freq=5)

In [0]:
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

def rebatch(pad_idx, batch):
    "Fix order in torchtext to match ours"
    src, trg = batch.src.transpose(0, 1), batch.trg.transpose(0, 1)
    return Batch(src, trg, pad_idx)

In [37]:
pad_idx = TGT.vocab.stoi["<blank>"]
model = make_model(len(SRC.vocab), len(TGT.vocab), N=6)
model.cuda()
criterion = LabelSmoothing(size=len(TGT.vocab), padding_idx=pad_idx, smoothing=0.1)
criterion.cuda()
BATCH_SIZE = 1024
train_iter = MyIterator(train, batch_size=BATCH_SIZE, device=0,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=True)
valid_iter = MyIterator(valid, batch_size=BATCH_SIZE, device=0,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=False)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [38]:
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 2000, torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
for epoch in range(3):
    model.train()
    run_epoch((rebatch(pad_idx, b) for b in train_iter), 
              model, SimpleLossCompute(model.generator, criterion, model_opt), None)
    #model.eval()
    #loss = run_epoch((rebatch(pad_idx, b) for b in valid_iter), 
     #                 model, SimpleLossCompute(model.generator, criterion, model_opt), None)
    #print(loss)


Epoch Step: 1 Loss: 8.666935 Tokens per Sec: 1482.072510
Save model and optimizer. Chpoint step #1
Epoch Step: 51 Loss: 7.961140 Tokens per Sec: 3730.075928
Epoch Step: 101 Loss: 7.244123 Tokens per Sec: 5313.408203
Epoch Step: 151 Loss: 5.896666 Tokens per Sec: 6082.214844
Epoch Step: 201 Loss: 5.673822 Tokens per Sec: 5535.458984
Epoch Step: 251 Loss: 5.648227 Tokens per Sec: 6038.280762
Epoch Step: 301 Loss: 5.630261 Tokens per Sec: 5312.396973
Epoch Step: 351 Loss: 5.377320 Tokens per Sec: 5871.135254
Epoch Step: 401 Loss: 5.236913 Tokens per Sec: 5823.162598
Epoch Step: 451 Loss: 5.318851 Tokens per Sec: 6085.551758
Epoch Step: 501 Loss: 5.131422 Tokens per Sec: 5955.096680
Save model and optimizer. Chpoint step #501
Epoch Step: 551 Loss: 4.975114 Tokens per Sec: 3824.022949
Epoch Step: 601 Loss: 4.922956 Tokens per Sec: 5060.404785
Epoch Step: 651 Loss: 4.940944 Tokens per Sec: 5222.835449
Epoch Step: 701 Loss: 4.697440 Tokens per Sec: 5939.000000
Epoch Step: 751 Loss: 4.821412 T

In [0]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

In [0]:
def beam_decode(model, src, src_mask, max_len, start_symbol, k = 5):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        if i == 0:
          out = model.decode(memory, src_mask, 
                            Variable(ys), 
                            Variable(subsequent_mask(ys.size(1))
                                      .type_as(src.data)))
          prob = model.generator(out[:, -1])
          next_probs, mt_index = prob.topk(k)
          next_words = [torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(mt_index[0][i])], dim=1) for i in range(k)]
         
        
        else:
          probs = []
          indexs = []
          for ind, word in enumerate(next_words):
            out = model.decode(memory, src_mask, 
                            Variable(word), 
                            Variable(subsequent_mask(word.size(1))
                                      .type_as(src.data)))
            prob = model.generator(out[:, -1])
            mt_prob, mt_index = prob.topk(k)
            probs.extend([mt_prob[0] + next_probs[0][ind]])
            indexs.extend([torch.cat([word, torch.ones(1, 1).type_as(src.data).fill_(mt_index[0][i])], dim=1) for i in range(k)])

          next_probs, indxs = torch.cat(probs, dim=0).topk(k)
          next_words = [indexs[indxs[i]] for i in range(k)]
          next_probs = next_probs.unsqueeze(0)
            
    _, mt_in = next_probs.topk(1)
    return next_words[mt_in[0]]

In [44]:
references, hypotheses = [], []
for i, batch in enumerate(valid_iter):
    for j in range(len(batch)-1):
      src = batch.src.transpose(0, 1)[j:j+1].cuda()
      src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2).cuda()
      out = beam_decode(model, src, src_mask, 
                          max_len=30, start_symbol=TGT.vocab.stoi["<s>"])
      
      print("Source:", end="\t")
      for i in range(1, out.size(1)+1):
          sym = SRC.vocab.itos[src[0, i]]
          if sym == "</s>": break
          print(sym, end ="")
      print()
      print("Translation:", end="\t")
      sentence_h = []
      for i in range(1, out.size(1)):
          sym = TGT.vocab.itos[out[0, i]]
          sentence_h.append(sym)
          if sym == "</s>": break
          print(sym, end ="")
      print()
      sentence_t = []
      print("Target:", end="\t")
      for i in range(1, batch.trg.size(0)):
          sym = TGT.vocab.itos[batch.trg.transpose(0, 1)[j:j+1, i]]
          sentence_t.append(sym)
          if sym == "</s>": break
          print(sym, end ="")
      print('\n')
    references.append(sentence_t)
    hypotheses.append(sentence_h)

Source:	▁как▁и▁палестинцы,▁курды▁достойны▁своего▁государства
Translation:	▁hezbollah▁and▁muslim▁muslims,▁muslims,▁muslims,▁muslims,▁and▁muslim
Target:	▁like▁the▁palestinians,▁the▁kurds▁deserve▁a▁state

Source:	▁напротив,▁она▁привела▁лишь▁к▁ухудшению▁ситуации.
Translation:	▁we▁have▁been▁able▁to▁do.
Target:	▁on▁the▁contrary,▁they▁have▁tended▁to▁make▁matters▁worse.

Source:	▁в▁случае▁же▁пакистана▁такие▁ожидания▁не▁оправданы.
Translation:	▁on▁the▁other▁hand,▁some▁pakistan’s▁president▁george▁w.▁bush’s▁president▁
Target:	▁with▁pakistan,▁no▁such▁assumption▁can▁be▁made.

Source:	▁почему▁бэннон▁должен▁был▁уйти
Translation:	▁washington,▁dc,▁john▁maynard▁keynes,▁john▁mccain
Target:	▁why▁bannon▁had▁to▁go

Source:	▁как▁может▁такая▁богатая▁страна▁оказаться▁так▁низко?
Translation:	▁will▁countries▁be▁able▁to▁be▁able▁to▁do?
Target:	▁how▁could▁such▁a▁rich▁country▁score▁so▁low?

Source:	▁что▁будет▁с▁многосторонней▁торговой▁системой?
Translation:	▁what▁will▁the▁world▁war▁ii?
Target:	▁wh<pad>▁the▁multilate

In [45]:
from nltk import translate
from nltk.translate.bleu_score import corpus_bleu
corpus_bleu(references, hypotheses, 
            smoothing_function=translate.bleu_score.SmoothingFunction().method3,
            auto_reweigh=True
           )

0.009098736995962125

with greedy search

In [46]:
references, hypotheses = [], []
for i, batch in enumerate(valid_iter):
    for j in range(len(batch)-1):
      src = batch.src.transpose(0, 1)[j:j+1].cuda()
      src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2).cuda()
      out = greedy_decode(model, src, src_mask, 
                          max_len=10, start_symbol=TGT.vocab.stoi["<s>"])
      
      print("Source:", end="\t")
      for i in range(1, out.size(1)+1):
          sym = SRC.vocab.itos[src[0, i]]
          if sym == "</s>": break
          print(sym, end ="")
      print()
      print("Translation:", end="\t")
      sentence_h = []
      for i in range(1, out.size(1)):
          sym = TGT.vocab.itos[out[0, i]]
          sentence_h.append(sym)
          if sym == "</s>": break
          print(sym, end ="")
      print()
      sentence_t = []
      print("Target:", end="\t")
      for i in range(1, batch.trg.size(0)):
          sym = TGT.vocab.itos[batch.trg.transpose(0, 1)[j:j+1, i]]
          sentence_t.append(sym)
          if sym == "</s>": break
          print(sym, end ="")
      print('\n')
    references.append(sentence_t)
    hypotheses.append(sentence_h)

Source:	▁как▁и▁палестинцы,▁курды▁достойны▁своего▁государства
Translation:	▁while▁the▁palestinian▁state▁and
Target:	▁like▁the▁palestinians,▁the▁kurds▁deserve▁a▁state

Source:	▁напротив,▁она▁привела▁лишь▁к▁ухудшению▁ситуации.
Translation:	▁we▁are▁now▁in▁the▁past.
Target:	▁on▁the▁contrary,▁they▁have▁tended▁to▁make▁matters▁worse.

Source:	▁в▁случае▁же▁пакистана▁такие▁ожидания▁не▁оправданы.
Translation:	▁with▁some▁of▁america’s▁leaders
Target:	▁with▁pakistan,▁no▁such▁assumption▁can▁be▁made.

Source:	▁почему▁бэннон▁должен▁был▁уйти
Translation:	▁why▁should▁be▁the▁world
Target:	▁why▁bannon▁had▁to▁go

Source:	▁как▁может▁такая▁богатая▁страна▁оказаться▁так▁низко?
Translation:	▁will▁you▁want▁to▁be▁a▁country
Target:	▁how▁could▁such▁a▁rich▁country▁score▁so▁low?

Source:	▁что▁будет▁с▁многосторонней▁торговой▁системой?
Translation:	▁what▁will▁be▁done?
Target:	▁wh<pad>▁the▁multilateral▁trading▁system?

Source:	▁готовиться▁к▁неизвестным▁неизвестностям
Translation:	▁greece’s▁leaders▁are▁now▁the
Target:	▁pr

In [47]:
from nltk import translate
from nltk.translate.bleu_score import corpus_bleu
corpus_bleu(references, hypotheses, 
            smoothing_function=translate.bleu_score.SmoothingFunction().method3,
            auto_reweigh=True
           )

0.020881972258231973